In [112]:

import dask.dataframe as dd
import numpy as np
from distributed import Client
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix, hstack

In [113]:
client = Client('127.0.0.1:8786')

In [114]:
df = dd.read_csv('/Users/marvin/testcases/datarevenue/data_root/raw/wine_dataset.csv', blocksize=1e6)
df = df.set_index('Unnamed: 0')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/marvin/testcases/datarevenue/data_root/raw/wine_dataset.csv'

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/usr/local/lib/python3.7/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/usr/local/lib/python3.7/site-packages/distributed/client.py", line 1268, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File

In [111]:
def clean_text(line):
    line = line.lower()
    return line

df['description'] = df['description'].apply(clean_text, meta=('description', 'str'))
df['description_len'] = df['description'].apply(len, meta=('description', 'str'))
df['country'] = df['country'].fillna('unk')


In [91]:
df['country'] = df['country'].astype('category')
df = df.categorize(columns=['country'])

In [71]:
vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
vectorizer.fit(ddf['description'])
description_encodings = vectorizer.transform(ddf['description'])

In [94]:
description_encodings

<10000x500 sparse matrix of type '<class 'numpy.float64'>'
	with 165033 stored elements in Compressed Sparse Row format>

In [97]:
country_encodings = csr_matrix(dd.get_dummies(df['country'], prefix = 'country'))

In [108]:
hstack((description_encodings, country_encodings)).todense()[0]

matrix([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.12876885, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.23087474, 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.20648877,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0

In [109]:
csr_matrix(ddf[['price', 'description_len']])

<1x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 10000 stored elements in Compressed Sparse Row format>

In [44]:
import numpy as np
import dask.bag as db

from toolz.curried import frequencies
from toolz.curried import valmap
from toolz.curried import merge_with
from toolz.curried import unique
from toolz import merge_with as _merge_with

In [55]:
def normalize(_dict):
    return {k:v/sum(_dict.values()) for k,v in _dict.items()}

corpus = 2*[
    "a a b b b z",
    "a a a a c z",
    "c b a d e f g z",
    "a b b b c c c d d d a b z",
    "c b a d e f g z",
    "a b b b c c c d d d a b z z z z z z z z z z z z z"
]


base = (
    db
    .from_sequence(corpus, npartitions=4)
    .map(str.split)
)

tf = (
    base
    .map(frequencies)
    .map(normalize)
)

idf = (
    base
    .map(unique)
    .map(frequencies)
    .reduction(merge_with(sum), merge_with(sum), split_every=2)
    .apply(normalize)
    .apply(valmap(np.log10))
    .apply(valmap(np.negative))
)

In [107]:
tf.compute()

[{'a': 0.3333333333333333, 'b': 0.5, 'z': 0.16666666666666666},
 {'a': 0.6666666666666666, 'c': 0.16666666666666666, 'z': 0.16666666666666666},
 {'c': 0.125,
  'b': 0.125,
  'a': 0.125,
  'd': 0.125,
  'e': 0.125,
  'f': 0.125,
  'g': 0.125,
  'z': 0.125},
 {'a': 0.15384615384615385,
  'b': 0.3076923076923077,
  'c': 0.23076923076923078,
  'd': 0.23076923076923078,
  'z': 0.07692307692307693},
 {'c': 0.125,
  'b': 0.125,
  'a': 0.125,
  'd': 0.125,
  'e': 0.125,
  'f': 0.125,
  'g': 0.125,
  'z': 0.125},
 {'a': 0.08, 'b': 0.16, 'c': 0.12, 'd': 0.12, 'z': 0.52},
 {'a': 0.3333333333333333, 'b': 0.5, 'z': 0.16666666666666666},
 {'a': 0.6666666666666666, 'c': 0.16666666666666666, 'z': 0.16666666666666666},
 {'c': 0.125,
  'b': 0.125,
  'a': 0.125,
  'd': 0.125,
  'e': 0.125,
  'f': 0.125,
  'g': 0.125,
  'z': 0.125},
 {'a': 0.15384615384615385,
  'b': 0.3076923076923077,
  'c': 0.23076923076923078,
  'd': 0.23076923076923078,
  'z': 0.07692307692307693},
 {'c': 0.125,
  'b': 0.125,
  'a': 

In [56]:
idf.compute()

{'a': 0.7269987279362623,
 'b': 0.8061799739838872,
 'z': 0.7269987279362623,
 'c': 0.8061799739838872,
 'd': 0.9030899869919435,
 'e': 1.2041199826559248,
 'f': 1.2041199826559248,
 'g': 1.2041199826559248}